# OSM PBF Loader

`OSMPbfLoader` can really quickly parse full OSM extract in the form of `*.osm.pbf` file.

It can download and parse a lot of features much faster than the `OSMOnlineLoader`, but it's much more useful when a lot of different features are required at once (like when using predefined filters).

When only a single or few features are needed, `OSMOnlineLoader` might be a better choice, since `OSMPbfLoader` will use a full extract of all features in a given region and will have to iterate over all of them.

In [ ]:
from srai.loaders.osm_loaders.filters import HEX2VEC_FILTER
from srai.loaders.osm_loaders.filters.popular import get_popular_tags
from srai.loaders.osm_loaders import OSMPbfLoader
from srai.utils.geometry import buffer_geometry
from srai.utils.constants import WGS84_CRS

import osmnx as ox
from shapely.geometry import Point
import geopandas as gpd

## Using OSMPbfLoader to download data for a specific area

### Download all features from `HEX2VEC_FILTER` in Warsaw, Poland

In [ ]:
loader = OSMPbfLoader()
warsaw_gdf = ox.geocode_to_gdf("Warsaw, Poland")
warsaw_features_gdf = loader.load(warsaw_gdf, HEX2VEC_FILTER)
warsaw_features_gdf

### Plot features

Inspired by [`prettymaps`](https://github.com/marceloprates/prettymaps)

In [ ]:
clipped_features_gdf = warsaw_features_gdf.clip(warsaw_gdf.geometry.unary_union)

In [ ]:
ax = warsaw_gdf.plot(color="lavender", figsize=(10, 10))

# plot water
clipped_features_gdf.dropna(subset=["water", "waterway"], how="all").plot(
    ax=ax, color="deepskyblue"
)

# plot greenery
clipped_features_gdf[
    clipped_features_gdf["landuse"].isin(
        ["grass", "orchard", "flowerbed", "forest", "greenfield", "meadow"]
    )
].plot(ax=ax, color="mediumseagreen")

# plot amenities
clipped_features_gdf.dropna(subset=["amenity"], how="all").plot(
    ax=ax, color="dimgray", markersize=0.1
)

xmin, ymin, xmax, ymax = warsaw_gdf.total_bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.set_axis_off()

### Download all features from popular tags based on OSMTagInfo in Vienna, Austria

In [ ]:
popular_tags = get_popular_tags(in_wiki_only=True)

num_keys = len(popular_tags)
f"Unique keys: {num_keys}."

In [ ]:
{k: popular_tags[k] for k in list(popular_tags)[:10]}

In [ ]:
vienna_center_circle = buffer_geometry(Point(16.37009, 48.20931), meters=1000)
vienna_center_circle_gdf = gpd.GeoDataFrame(geometry=[vienna_center_circle], crs=WGS84_CRS)

In [ ]:
loader = OSMPbfLoader()
vienna_features_gdf = loader.load(vienna_center_circle_gdf, popular_tags)
vienna_features_gdf

### Plot features

In [ ]:
clipped_vienna_features_gdf = vienna_features_gdf.clip(vienna_center_circle)

In [ ]:
ax = vienna_center_circle_gdf.plot(color="#F2F4CB", figsize=(10, 10))

# plot water
clipped_vienna_features_gdf.dropna(subset=["water", "waterway"], how="all").plot(
    ax=ax, color="deepskyblue"
)

# plot streets
clipped_vienna_features_gdf.dropna(subset=["highway"], how="all").plot(
    ax=ax, color="#475657", markersize=0.1
)

# plot buildings
clipped_vienna_features_gdf.dropna(subset=["building"], how="all").plot(ax=ax, color="#FF5E5B")

# plot parkings
clipped_vienna_features_gdf[
    (clipped_vienna_features_gdf["amenity"] == "parking")
    | (clipped_vienna_features_gdf["highway"] == "pedestrian")
].plot(ax=ax, color="#2F3737", markersize=0.1)

# plot greenery
clipped_vienna_features_gdf[
    clipped_vienna_features_gdf["landuse"].isin(
        ["grass", "orchard", "flowerbed", "forest", "greenfield", "meadow"]
    )
].plot(ax=ax, color="#8BB174")

xmin, ymin, xmax, ymax = vienna_center_circle_gdf.total_bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.set_axis_off()